In [1]:
from pydap.client import open_url
from pydap.cas.urs import setup_session
import datetime as dt  # Python standard library datetime  module
import numpy as np
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, addcyclic, shiftgrid
import pandas as pd
from datetime import datetime
from datetime import timedelta

# Global Vars
BASE_URL = 'E:/DataForDS/NCALDAS_NOAH0125_D.A'
END_URL = '.002.txt'

In [2]:
#Read Fire data
data = pd.read_csv("Top1000FireLocationsCases.csv")
data = data.drop(columns=["Unnamed: 0","Unnamed: 0.1","STAT_CAUSE_DESCR"])
data = data.sort_values(by ='DISCOVERY_DATE')
data

,DISCOVERY_DATE,LATITUDE,LONGITUDE,X_tile,Y_tile
299776,1992-01-02,32.972600,-83.233900,63,334
60692,1992-01-02,33.558333,-79.945833,68,360
322072,1992-01-02,33.062500,-80.379167,64,356
129139,1992-01-02,30.193750,-97.322910,41,221
281930,1992-01-02,33.920833,-80.279167,71,357
...,...,...,...,...,...
125857,2016-01-01,32.439411,-94.947366,59,240
150313,2016-01-01,33.511928,-112.147062,68,102
110200,2016-01-01,33.470948,-112.024130,67,103
102925,2016-01-01,26.596500,-81.739600,12,346


In [3]:
import random

myDateFormat = '%Y-%m-%d'
lats = np.genfromtxt('NoahLats.csv', delimiter=',')
lons = np.genfromtxt('NoahLons.csv', delimiter=',')

In [4]:
#Dis = map(lambda x: datetime.strptime(x,myDateFormat),data['DISCOVERY_DATE'])
import gc
import timeit
import random
import math


start = timeit.default_timer()

i=0
oldDate = None
dataset = None


Tair_f= []
Tair_f_min= []    
Tair_f_max= []
Wind_f= []
DOY = []
Humidity = []

for indexes,fire in data.iterrows():
    date = datetime.strptime(str(fire['DISCOVERY_DATE']),myDateFormat)
    year = str(date.year)
    month = '{:02d}'.format(date.month)
    day = '{:02d}'.format(date.day)
    lat_idx = fire['X_tile']
    lon_idx = fire['Y_tile']
    
    try:
        if (date != oldDate):
            if (dataset != None):
                dataset.close()
            FullURL = BASE_URL+year+month+day+END_URL
            dataset = Dataset(FullURL, 'r')
    
        q = dataset.variables['Qair_f'][:][0,lat_idx,lon_idx].data.tolist()
        p = dataset.variables['Psurf_f'][:][0,lat_idx,lon_idx].data.tolist()
        T = dataset.variables['Tair_f'][:][0,lat_idx,lon_idx].data.tolist()
        DOY.append(date.timetuple().tm_yday)
        Tair_f.append(T)
        Tair_f_min.append(dataset.variables['Tair_f_min'][:][0,lat_idx,lon_idx].data.tolist())   
        Tair_f_max.append(dataset.variables['Tair_f_max'][:][0,lat_idx,lon_idx].data.tolist())
        Wind_f.append(dataset.variables['Wind_f'][:][0,lat_idx,lon_idx].data.tolist())
        Humidity.append((0.263 * p * q )/(math.exp((17.67* (T - 273.16 ))/(T-29.65))))
        
        i=i+1
        if (i%10000 == 0):
            stop = timeit.default_timer()
            print('Time: ', stop - start , (i/len(data))*100,"% is Done")
            start = timeit.default_timer()
        oldDate = date
    except:
        DOY.append(np.NaN)
        Tair_f.append(np.NaN)
        Tair_f_min.append(np.NaN)   
        Tair_f_max.append(np.NaN)
        Wind_f.append(np.NaN)
        Humidity.append(np.NaN) 

data['Tair_f']=Tair_f
data['Tair_f_min']=Tair_f_min
data['Tair_f_max']=Tair_f_max
data['Wind_f']=Wind_f
data['DOY']=DOY
data['Humidity']=Humidity

data

Time:  70.8958419 3.014699675618315 % is Done
Time:  95.6478187 6.02939935123663 % is Done
Time:  88.33373350000002 9.044099026854944 % is Done
Time:  81.21334200000001 12.05879870247326 % is Done
Time:  91.38874390000001 15.073498378091573 % is Done
Time:  81.20819280000006 18.08819805370989 % is Done
Time:  91.51391449999994 21.102897729328205 % is Done
Time:  82.79467270000009 24.11759740494652 % is Done
Time:  107.24954660000003 27.132297080564832 % is Done
Time:  92.81384070000001 30.146996756183146 % is Done
Time:  86.56162649999999 33.16169643180146 % is Done
Time:  67.76415869999994 36.17639610741978 % is Done
Time:  91.64317959999994 39.191095783038094 % is Done
Time:  88.2563841000001 42.20579545865641 % is Done
Time:  75.82307029999993 45.22049513427472 % is Done
Time:  112.32978700000012 48.23519480989304 % is Done
Time:  100.10056229999986 51.24989448551135 % is Done
Time:  90.58216330000005 54.264594161129665 % is Done
Time:  68.96399840000004 57.27929383674798 % is Done


,DISCOVERY_DATE,LATITUDE,LONGITUDE,X_tile,Y_tile,Tair_f,Tair_f_min,Tair_f_max,Wind_f,DOY,Humidity
299776,1992-01-02,32.972600,-83.233900,63,334,281.790680,278.930023,284.580017,4.763154,2.0,91.497999
60692,1992-01-02,33.558333,-79.945833,68,360,285.787903,283.302551,289.670013,6.953478,2.0,89.305454
322072,1992-01-02,33.062500,-80.379167,64,356,286.742310,284.582489,290.059875,5.878381,2.0,88.981029
129139,1992-01-02,30.193750,-97.322910,41,221,287.292725,285.539948,290.719971,2.052834,2.0,82.723779
281930,1992-01-02,33.920833,-80.279167,71,357,284.059784,281.600006,287.940033,5.527092,2.0,86.945962
...,...,...,...,...,...,...,...,...,...,...,...
125857,2016-01-01,32.439411,-94.947366,59,240,NaN,NaN,NaN,NaN,NaN,NaN
150313,2016-01-01,33.511928,-112.147062,68,102,NaN,NaN,NaN,NaN,NaN,NaN
110200,2016-01-01,33.470948,-112.024130,67,103,NaN,NaN,NaN,NaN,NaN,NaN
102925,2016-01-01,26.596500,-81.739600,12,346,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
data.dropna(inplace=True)
data.to_csv('C:/Fires/YesFiresNoToArson.csv',mode ='a',index=False)